In [13]:
import arcpy
import heapq
import math
from collections import defaultdict

arcpy.env.workspace = r"C:\Users\Karolina\Desktop\Studia\Semestr_5\PAG\P1_PAG\Projekt1_PAG\Projekt1_PAG.gdb"
arcpy.env.overwriteOutput = True


# read shapefile where FID < 20
roads_shapefile = r"C:\Users\Karolina\Desktop\Studia\Semestr_5\PAG\P1_PAG\Projekt1_PAG\BDOT_Torun\L4_1_BDOT10k__OT_SKJZ_L.shp"
vertices_file = r"C:\Users\Karolina\Desktop\Studia\Semestr_5\PAG\P1_PAG\Projekt1_PAG\vertices.txt"
edges_file = r"C:\Users\Karolina\Desktop\Studia\Semestr_5\PAG\P1_PAG\Projekt1_PAG\edges.txt"

# Słownik do przechowywania unikalnych wierzchołków
vertices = {}
edges = []

# Create vertex if one doesn't exist with the same coords
def add_vertex(x, y, vertex_id):
    if (x, y) not in vertices:
        vertices[(x, y)] = {'id': vertex_id, 'x': x, 'y': y, 'edges': []}
    return vertices[(x, y)]['id']

def speed(klasa):
    kmh2ms = 1000 / 3600
    if klasa == 'A':
        return 140 * kmh2ms
    elif klasa == 'S':
        return 120 * kmh2ms
    elif klasa == 'GP':
        return 70 * kmh2ms
    elif klasa == 'G':
        return 60 * kmh2ms
    elif klasa == 'Z':
        return 50 * kmh2ms
    elif klasa == 'L':
        return 40 * kmh2ms
    elif klasa == 'D':
        return 20 * kmh2ms
    elif klasa == 'I':
        return 20 * kmh2ms
    return 0

# Read road shp
with arcpy.da.SearchCursor(roads_shapefile, ["SHAPE@", "FID", "klasaDrogi"]) as cursor:
    vertex_id = 0
    edge_id = 0

    for row in cursor:
        line = row[0]
        FID = row[1]
        klasa = row[2]
        speedms = speed(klasa)

        
        # read start and end point, round to one meter
        start_point = line.firstPoint
        end_point = line.lastPoint

        start_point = (round(start_point.X, 0), round(start_point.Y, 0))
        end_point = (round(end_point.X, 0), round(end_point.Y, 0))
        
        start_vertex = add_vertex(start_point[0], start_point[1], vertex_id)
        vertex_id += 1 if start_vertex == vertex_id else 0

        end_vertex = add_vertex(end_point[0], end_point[1], vertex_id)
        vertex_id += 1 if end_vertex == vertex_id else 0

        # add edges
        edges.append({
            "id": edge_id,
            "from": start_vertex,
            "to": end_vertex,
            "road_id": FID,
            "length": line.length,
            "speed": speedms
        })

        edge_id += 1
        
        # add edges to the vertex file
        vertices[start_point]['edges'].append(edge_id)
        vertices[end_point]['edges'].append(edge_id)


# Write vertices
with open(vertices_file, "w") as vf:
    vf.write("id\tx\ty\tedges\n")
    for v in vertices.values():
        vf.write(f"{v['id']}\t{v['x']}\t{v['y']}\t{','.join(map(str, v['edges']))}\n")

# Write edges
with open(edges_file, "w") as ef:
    ef.write("id\tfrom\tto\troad_id\tlength\tspeed\n")
    for edge in edges:
        ef.write(f"{edge['id']}\t{edge['from']}\t{edge['to']}\t{edge['road_id']}\t{edge['length']}\t{edge['speed']}\n")

print("Liczba wierzchołków:", len(vertices))
print("Liczba krawędzi:", len(edges))

Liczba wierzchołków: 9385
Liczba krawędzi: 11781


In [14]:
start = 2492
end = 1807

In [15]:
# Makes points from the vertex.txt file
arcpy.management.XYTableToPoint(
    in_table=vertices_file,
    out_feature_class=r"vertices",
    x_field="x",
    y_field="y",
    z_field=None,
    coordinate_system='PROJCS["ETRS_1989_UWPP_1992",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Gauss_Kruger"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",-5300000.0],PARAMETER["Central_Meridian",19.0],PARAMETER["Scale_Factor",0.9993],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5119200 -15295100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Users\\Karolina\\Desktop\\Studia\\Semestr_5\\PAG\\P1_PAG\\Projekt1_PAG\\Projekt1_PAG.gdb\\vertices'>

In [16]:
def read_graph_undirected(filename):
    f = open(filename)
    g = defaultdict(list)
    road_ids = {}
    road_lengths = {}
    road_speeds = {}
    
    f.readline()  # Pomijamy nagłówek
    for line in f:
        e = line.split()
        from_vertex, to_vertex, road_id = int(e[1]), int(e[2]), int(e[3])
        length, speed = float(e[4]), float(e[5])

        # Dodajemy krawędzi do grafu, z uwzględnieniem prędkości
        g[from_vertex].append((to_vertex, length, speed))  # Zawiera teraz długość i prędkość
        g[to_vertex].append((from_vertex, length, speed))  # Zawiera teraz długość i prędkość

        # Zapisujemy dane o drogach (id, długość, prędkość) w dwóch kierunkach (graf nieukierunkowany)
        road_ids[(from_vertex, to_vertex)] = road_id
        road_ids[(to_vertex, from_vertex)] = road_id
        road_lengths[(from_vertex, to_vertex)] = length
        road_lengths[(to_vertex, from_vertex)] = length
        road_speeds[(from_vertex, to_vertex)] = speed
        road_speeds[(to_vertex, from_vertex)] = speed
        
    return g, road_ids, road_lengths, road_speeds

def retrieve_path(prev, a, b, road_ids):
    path = [b]
    road_path = []
    while b != a:
        b = prev[b]
        path.append(b)
        road_path.append(road_ids[(b, path[-2])])
    path.reverse()
    road_path.reverse()
    return path, road_path

# Funkcja heurystyki - odległość euklidesowa podzielona przez maksymalną prędkość
def heuristic(v, goal, vertices, road_speeds):
    # Pobieramy współrzędne punktu początkowego i końcowego
    vx, vy = vertices[v]['x'], vertices[v]['y']
    gx, gy = vertices[goal]['x'], vertices[goal]['y']
    
    # Obliczamy odległość euklidesową
    distance = math.sqrt((vx - gx) ** 2 + (vy - gy) ** 2)
    
    # Szukamy maksymalnej prędkości na trasie
    max_speed = max(road_speeds.values())
    
    # Heurystyka to odległość euklidesowa podzielona przez maksymalną prędkość
    return distance / max_speed


# A* pathfinding algorithm, teraz przyjmuje heurystykę jako parametr
def a_star_path(graph, road_ids, road_speeds, vertices, start, end, heuristic_fn):
    open_set = []  # kolejka priorytetowa
    heapq.heappush(open_set, (0, start))  # (f-score, vertex)
    g_score = {vertex: float('inf') for vertex in graph}
    g_score[start] = 0
    prev = {}

    while open_set:
        # Pobieramy wierzchołek o najniższym koszcie f
        _, current = heapq.heappop(open_set)

        if current == end:
            return retrieve_path(prev, start, end, road_ids)

        # Iteracja po sąsiadach
        for neighbor, length, speed in graph[current]:
            tentative_g_score = g_score[current] + (length / speed)  # Czas przejazdu
            if tentative_g_score < g_score[neighbor]:
                prev[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score = g_score[neighbor] + heuristic_fn(neighbor, end, vertices, road_speeds)  # Obliczamy f-score
                heapq.heappush(open_set, (f_score, neighbor))

    return None

# Wczytanie grafu i danych
g, road_ids, road_lengths, road_speeds = read_graph_undirected(edges_file)

# Wczytanie wierzchołków
vertices_dict = {}
with open(vertices_file, 'r') as vf:
    vf.readline()  # Pomijamy nagłówek
    for line in vf:
        parts = line.split()
        v_id = int(parts[0])
        x, y = float(parts[1]), float(parts[2])
        vertices_dict[v_id] = {'x': x, 'y': y}

# Wywołanie algorytmu A* z funkcją heurystyki jako argumentem
path, road_path = a_star_path(g, road_ids, road_speeds, vertices_dict, start, end, heuristic)

# Wyświetlenie wyników
print("Trasa:", path)
print("ID dróg na trasie:", road_path)

# count total length and time
def convert_time(total_seconds):
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{int(hours)}h {int(minutes)}min {int(seconds)}sec"

def len_time(path):
    total_length = 0
    total_time = 0

    for i in range(len(path) - 1):
        from_vertex = path[i]
        to_vertex = path[i + 1]
        
        segment_length = road_lengths[(from_vertex, to_vertex)]
        segment_speed = road_speeds[(from_vertex, to_vertex)]
        
        total_length += segment_length
        total_time += segment_length / segment_speed
    
    return total_length, total_time

# Obliczenie całkowitej długości i czasu przejazdu
total_length, total_time = len_time(path)
formatted_time = convert_time(total_time)

print("Długość trasy:", round(total_length, 2), "metrów")
print("Całkowity czas:", formatted_time)

Trasa: [2492, 8630, 2503, 3153, 3154, 524, 8631, 8697, 7436, 2846, 8698, 2786, 2368, 1677, 2319, 2505, 1574, 2488, 1568, 8398, 7229, 9057, 8704, 1703, 1704, 8703, 8705, 8702, 137, 8701, 7501, 4483, 724, 8700, 3494, 6496, 3493, 3622, 4418, 7606, 8804, 4, 3, 8806, 8805, 8808, 8168, 8166, 5567, 5778, 6902, 6903, 6904, 6905, 6909, 6906, 6907, 6908, 7797, 8081, 8080, 3778, 8082, 5601, 2700, 8083, 8084, 9031, 3515, 9042, 6194, 6250, 9339, 9341, 9340, 7104, 7373, 7374, 7376, 5575, 7377, 7378, 2931, 2937, 4441, 7375, 7382, 7052, 5430, 7387, 7386, 8291, 8298, 3137, 8295, 5671, 9028, 9029, 7529, 7525, 6837, 3149, 6835, 3147, 6836, 3139, 3132, 5195, 6833, 6838, 5289, 7917, 2773, 330, 2426, 5900, 1855, 1251, 7568, 7569, 6656, 6648, 7247, 7905, 6644, 1841, 1830, 1839, 1112, 1836, 1835, 1832, 1807]
ID dróg na trasie: [9511, 9512, 9509, 9514, 9515, 9513, 9735, 9737, 9740, 9741, 9736, 9742, 9743, 11661, 11662, 9744, 9745, 9734, 9749, 9738, 11122, 11123, 9759, 9760, 9758, 9761, 9762, 9757, 9763, 9748, 

In [17]:
# Ścieżka do wyjściowego shapefile
output_shapefile = r"C:\Users\Karolina\Desktop\Studia\Semestr_5\PAG\P1_PAG\Projekt1_PAG\out_path_time.shp"

# Tworzymy nowy shapefile linii
arcpy.CreateFeatureclass_management(
    out_path=r"C:\Users\Karolina\Desktop\Studia\Semestr_5\PAG\P1_PAG\Projekt1_PAG",  # folder wyjściowy
    out_name="out_path_time.shp",  # nazwa shapefile
    geometry_type="POLYLINE",         # typ geometrii (linia)
    spatial_reference=arcpy.SpatialReference(2180)  # Używamy układu WGS 84 (EPSG:4326) - zmień na odpowiedni układ współrzędnych
)

# Dodajemy pola do shapefile'a
arcpy.AddField_management(output_shapefile, "road_id", "LONG")  # ID drogi
arcpy.AddField_management(output_shapefile, "length", "DOUBLE") # długość odcinka
arcpy.AddField_management(output_shapefile, "speed", "DOUBLE")  # prędkość na odcinku

# Funkcja zapisująca ścieżkę do shapefile'a
def save_path_to_shapefile(path, road_path, road_lengths, road_speeds, vertices):
    # Otwieramy inserter
    with arcpy.da.InsertCursor(output_shapefile, ["SHAPE@", "road_id", "length", "speed"]) as cursor:
        # Przechodzimy po parach wierzchołków na ścieżce
        for i in range(len(path) - 1):
            from_vertex = path[i]
            to_vertex = path[i + 1]
            
            # Pobieramy współrzędne punktów
            from_point = arcpy.Point(vertices[from_vertex]['x'], vertices[from_vertex]['y'])
            to_point = arcpy.Point(vertices[to_vertex]['x'], vertices[to_vertex]['y'])
            
            # Tworzymy obiekt linii
            line = arcpy.Polyline(arcpy.Array([from_point, to_point]))
            
            # Pobieramy dodatkowe atrybuty
            road_id = road_path[i]
            length = road_lengths[(from_vertex, to_vertex)]
            speed = road_speeds[(from_vertex, to_vertex)]
            
            # Wstawiamy linię do shapefile'a
            cursor.insertRow([line, road_id, length, speed])

# Wywołujemy funkcję zapisującą
save_path_to_shapefile(path, road_path, road_lengths, road_speeds, vertices_dict)

print("Ścieżka została zapisana do pliku shapefile:", output_shapefile)

Ścieżka została zapisana do pliku shapefile: C:\Users\Karolina\Desktop\Studia\Semestr_5\PAG\P1_PAG\Projekt1_PAG\out_path_time.shp
